In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
#Change the working directory
import os
import gc
os.chdir('/content/gdrive/My Drive/Colab Notebooks/FaceRecognition')
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
os.listdir()

['model',
 'Sharukha-1.jpg',
 'sharukhSalman.png',
 'Dataset',
 'CheckPoints',
 'FaceNet_J.ipynb',
 'output',
 '.ipynb_checkpoints',
 'debug.JPG',
 'Copy of FaceNet_J.ipynb']

In [3]:
os.listdir()

['model',
 'Sharukha-1.jpg',
 'sharukhSalman.png',
 'Dataset',
 'CheckPoints',
 'FaceNet_J.ipynb',
 'output',
 '.ipynb_checkpoints',
 'debug.JPG',
 'Copy of FaceNet_J.ipynb']

In [4]:
!sudo pip install mtcnn

     |████████████████████████████████| 2.3MB 5.0MB/s 
     |████████████████████████████████| 28.7MB 34.4MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 3.4.7.28
    Uninstalling opencv-python-3.4.7.28:
      Successfully uninstalled opencv-python-3.4.7.28


In [5]:
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

Using TensorFlow backend.


0.1.0


In [0]:
# demonstrate face detection on 5 Celebrity Faces Dataset
from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
 
# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
	face_array = list()
	# load image from file
	image = Image.open(filename)
	# convert to RGB, if needed
	image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	for resulti in results:
		x1, y1, width, height = resulti['box']
		# bug fix
		x1, y1 = abs(x1), abs(y1)
		x2, y2 = x1 + width, y1 + height
		# extract the face
		face = pixels[y1:y2, x1:x2]
		# resize pixels to the model size
		image = Image.fromarray(face)
		image = image.resize(required_size)
		face_array.append(asarray(image))
	return face_array
 
# specify folder to plot
folder = './Dataset/train/Salman/'
i = 1
# enumerate files
for filename in listdir(folder):
	# path
	#path = folder + filename
	path = '/content/gdrive/My Drive/Colab Notebooks/FaceRecognition/debug.JPG'
	# get face
	face = extract_face(path)
	for facei in face:
		#print(i, face.shape)
		# plot
		pyplot.subplot(2, 7, i)
		pyplot.axis('off')
		pyplot.imshow(facei)
		i += 1
		pyplot.show()

In [0]:
!pwd
results.shape()

In [6]:
# face detection for the 5 Celebrity Faces Dataset
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN
 
# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
	# load image from file
	image = Image.open(filename)
	# convert to RGB, if needed
	image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	# bug fix
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array
 
# load images and extract faces for all images in a directory
def load_faces(directory):
	faces = list()
	# enumerate files
	for filename in listdir(directory):
		# path
		path = directory + filename
		# get face
		face = extract_face(path)
		# store
		faces.append(face)
	return faces
 
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
	X, y = list(), list()
	# enumerate folders, on per class
	for subdir in listdir(directory):
		# path
		path = directory + subdir + '/'
		# skip any files that might be in the dir
		if not isdir(path):
			continue
		# load all faces in the subdirectory
		faces = load_faces(path)
		# create labels
		labels = [subdir for _ in range(len(faces))]
		# summarize progress
		print('>loaded %d examples for class: %s' % (len(faces), subdir))
		# store
		X.extend(faces)
		y.extend(labels)
	return asarray(X), asarray(y)
 
# load train dataset

trainX, trainy = load_dataset('./Dataset/train/')
print(trainX.shape, trainy.shape)
# load test dataset
testX, testy = load_dataset('./Dataset/val/')
# save arrays to one file in compressed format
savez_compressed('./CheckPoints/5-celebrity-faces-dataset.npz', trainX, trainy, testX, testy)











>loaded 7 examples for class: Mohanlal
>loaded 7 examples for class: Rajani
>loaded 8 examples for class: Chiranjeevi
>loaded 7 examples for class: Sharukh
>loaded 5 examples for class: Salman
>loaded 7 examples for class: SundarPichai
>loaded 5 examples for class: Alen
>loaded 7 examples for class: Elisa
>loaded 6 examples for class: Wangyi
>loaded 4 examples for class: Michale
>loaded 9 examples for class: EMP_01
>loaded 4 examples for class: Denis
(76, 160, 160, 3) (76,)
>loaded 4 examples for class: Chiranjeevi
>loaded 4 examples for class: Sharukh
>loaded 2 examples for class: Rajani
>loaded 6 examples for class: Salman
>loaded 2 examples for class: Mohanlal


Run the below cell when the new dataset is added

In [7]:
# calculate a face embedding for each face in the dataset using facenet
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
 
# get the face embedding for one face
def get_embedding(model, face_pixels):
	# scale pixel values
	face_pixels = face_pixels.astype('float32')
	# standardize pixel values across channels (global)
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	# transform face into one sample
	samples = expand_dims(face_pixels, axis=0)
	# make prediction to get embedding
	yhat = model.predict(samples)
	return yhat[0]
 
# load the face dataset
data = load('./CheckPoints/5-celebrity-faces-dataset.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
# load the facenet model
model = load_model('./model/facenet_keras.h5')
print('Loaded Model')
# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
	embedding = get_embedding(model, face_pixels)
	newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
	embedding = get_embedding(model, face_pixels)
	newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)
# save arrays to one file in compressed format
savez_compressed('././CheckPoints/5-celebrity-faces-embeddings.npz', newTrainX, trainy, newTestX, testy)

Loaded:  (76, 160, 160, 3) (76,) (18, 160, 160, 3) (18,)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded Model
(76, 128)
(18, 128)


Below code block is not required

In [0]:
# develop a classifier for the 5 Celebrity Faces Dataset
from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
# load dataset
data = load('./CheckPoints/5-celebrity-faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)
# predict
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

In [0]:
from imutils import face_utils
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
from numpy import savez_compressed
from mtcnn.mtcnn import MTCNN

detector = MTCNN()

def resize_face(face_pixels, required_size=(160, 160)):
		# resize pixels to the model size
		image = Image.fromarray(face_pixels)
		image = image.resize(required_size)
		face_array=asarray(image)
		return face_array

def get_embedding(FRmodel, img_pixels):
  #face_resize = ()
  required_size=(160, 160)
  #print(img_pixels.shape)
	# scale pixel values
  face_pixels = resize_face(img_pixels)
  face_pixels = face_pixels.astype('float32')
  #print(face_pixels.shape)
	# standardize pixel values across channels (global)
  mean, std = face_pixels.mean(), face_pixels.std()
  face_pixels = (face_pixels - mean) / std
	# transform face into one sample
  samples = expand_dims(face_pixels, axis=0)
  # make prediction to get embedding
  yhat = FRmodel.predict(samples)
  return yhat[0]

def recognize_face(FRmodel, face_pixels):
    data = load('./CheckPoints/5-celebrity-faces-embeddings.npz')
    trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
    face_embedding = get_embedding(FRmodel, face_pixels)
    #print(face_embedding.shape)
    min_dist = 100
    identity = None

    # normalize input vectors
    in_encoder = Normalizer(norm='l2')    
    #face_embedding = in_encoder.transform(face_embedding)
    samples = expand_dims(face_embedding, axis=0)
    # label encode targets
    out_encoder = LabelEncoder()
    out_encoder.fit(trainy)
    trainy = out_encoder.transform(trainy)
    # fit model
    model = SVC(kernel='linear', probability=True)
    model.fit(trainX, trainy)
    yhat_class = model.predict(samples)
    yhat_prob = model.predict_proba(samples)
    # get name
    class_index = yhat_class[0]
    #print(class_index)
    class_probability = yhat_prob[0,class_index] * 100
    predict_names = out_encoder.inverse_transform(yhat_class)
    #print(predict_names[0])
    #print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
    #print('Expected: %s' % random_face_name[0])
    return predict_names[0],class_probability
    

def extract_face(img_rgb, required_size=(160, 160)):
	face_array = list()
	# load image from file
	#image = Image.open(filename)
	#image = image.convert('RGB')
	# convert to array
	pixels = asarray(img_rgb)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	for resulti in results:
		x1, y1, width, height = resulti['box']
		# bug fix
		x1, y1 = abs(x1), abs(y1)
		x2, y2 = x1 + width, y1 + height
		# extract the face
		face = pixels[y1:y2, x1:x2]
		# resize pixels to the model size
		image = Image.fromarray(face)
		image = image.resize(required_size)
		face_array.append(asarray(image))
	return face_array

def extract_face_info(img, img_rgb, FRmodel):
    faces = detector.detect_faces(img_rgb)
    x, y, w, h = 0, 0, 0, 0
    if len(faces) > 0:
        for face in faces:
            #print(face.shape)
            #(x, y, w, h) = face_utils.rect_to_bb(face)
            (x, y, w, h) = face['box']
            #print("jani1")
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 0), 2)
            #print("jani2")
            image = img[y:y + h, x:x + w]
            name, Prob = recognize_face(FRmodel, image)
            #print(name)
            #print(Prob)
            if Prob > 40:
                #cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 0), 2)
                cv2.putText(img, "Face : " + name, (x, y - 50), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
                cv2.putText(img, "Dist : " + str(Prob), (x, y - 20), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
            #else:
                #cv2.putText(img, 'No matching faces', (x, y - 20), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255), 2)

In [9]:
# import the necessary packages
from random import choice
from numpy import load
from keras.models import load_model
from numpy import expand_dims
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot

import cv2
from imutils import face_utils
import dlib
from google.colab.patches import cv2_imshow

data = load('./CheckPoints/5-celebrity-faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
FRmodel = load_model('./model/facenet_keras.h5')
print('Loaded Model')
#cap = cv2.VideoCapture(0)

# initialize the video stream, pointer to output video file, and
# frame dimensions
#database = initialize()
cap = cv2.VideoCapture("/content/gdrive/My Drive/Colab Notebooks/FaceRecogn_Video/Dataset/Manufacturing_Industry_1.mp4")
writer = None
(W, H) = (None, None)
 
# try to determine the total number of frames in the video file
try:
	prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
		else cv2.CAP_PROP_FRAME_COUNT
	total = int(cap.get(prop))
	print("[INFO] {} total frames in video".format(total))
 
# an error occurred while trying to determine the total
# number of frames in the video file
except:
	print("[INFO] could not determine # of frames in video")
	print("[INFO] no approx. completion time can be provided")
	total = -1
  
while True:
    # load the input image and convert it to grayscale
    ret, img = cap.read()
    if not ret:
        break
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    # detect faces in the grayscale image
    try:
      extract_face_info(img, img_rgb, FRmodel)            
    except ValueError:
      pass

    if writer is None:
        # initialize our video writer
		    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		    writer = cv2.VideoWriter("/content/gdrive/My Drive/Colab Notebooks/FaceRecognition/output/Manufacturing_Industry_1_Output.mp4", fourcc, 30,(img.shape[1], img.shape[0]), True)

    writer.write(img)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

writer.release()
cv2.destroyAllWindows()
cap.release()
print("Predictions saved to output directory")

Loaded Model
[INFO] could not determine # of frames in video
[INFO] no approx. completion time can be provided
Predictions saved to output directory
